# Uttilisation du fichier Yaml avec python

## Read YAML file 

**Mieux vaut uttiliser yal.self_load() plutot qe yaml.load() , c'est plus sécurisé car la fonction self_load() nous protege contre les vulnérabilitées d'execution de code arbitraire**

In [3]:
# En executant cette cellule , nous obtenons un dictionnaire Python affiché, sans erreur .
import yaml

with open('config.yaml', 'r') as f:
    data = yaml.safe_load(f)          # Uttilsation de safe_load() pour parser le contenu en un dictionnaire python
print(data)


{'database': {'host': 'localhost', 'port': 5432, 'credentials': {'username': 'admin', 'password': 'secret'}}}


## Génération de fichiers de YAML 

**IL est possible de gererer notre fichier YAML a l'aide de quelques lignes de code python**

In [ ]:
import yaml

# Création d'un dictionaire python imbriqué avec notre  configuation 
config = {
    'database': {
        'host': 'localhost',
        'port': 5432,
        'name': 'myapp_db',
        'credentials': {
            'username': 'admin',
            'password': 'secret123'
        }
    },
    'server': {
        'host': '0.0.0.0',
        'port': 8000,
        'debug': True
    },
    'features': {
        'enable_cache': True,
        'cache_ttl': 3600
    }
}

# Ecriture du fichier YAML
with open('generated_config.yaml', 'w') as file:      # ouverture du fichier en mode écriture :'w'
    yaml.dump(config, file, default_flow_style=False) # convertit le dictionaire python au format YAML


## Manipuler les listes avec YAML

**YAML gere les les listes avec élégance, prennons  exemple  la gestion d'une liste de points de terminaisons d'api**

In [ ]:
import yaml

# Configuration avec  listes
services_config = {
    'services': [
        {
            'name': 'auth-service',
            'url': 'http://auth.example.com',
            'timeout': 30
        },
        {
            'name': 'payment-service',
            'url': 'http://payment.example.com',
            'timeout': 60
        },
        {
            'name': 'notification-service',
            'url': 'http://notification.example.com',
            'timeout': 15
        }
    ],
    'retry_policy': {
        'max_attempts': 3,
        'backoff_seconds': 5
    }
}

# Ecriture du document
with open('services.yaml', 'w') as file:
    yaml.dump(services_config, file, default_flow_style=False, sort_keys=False) # Sort_keys=False préserve l'odre des clés tel que défini

# Lecture
with open('services.yaml', 'r') as file:
    loaded_services = yaml.safe_load(file)

# Access aux  items de la liste
for service in loaded_services['services']:
    print(f"Service: {service['name']}, URL: {service['url']}")


Service: auth-service, URL: http://auth.example.com
Service: payment-service, URL: http://payment.example.com
Service: notification-service, URL: http://notification.example.com


## Creer un gestionaire de configuration YAML 

In [ ]:
import yaml
import os

# Initialisation
class ConfigManager:
    def __init__(self, config_dir='configs'):
        self.config_dir = config_dir
        self.config = {}

    def load_config(self, environment='development'):
        """Load configuration for a specific environment"""
        config_file = os.path.join(self.config_dir, f'{environment}.yaml')

        try:
            with open(config_file, 'r') as file:
                self.config = yaml.safe_load(file)
            print(f"✓ Loaded configuration for {environment}")
            return self.config
        except FileNotFoundError:
            print(f"✗ Configuration file not found: {config_file}")
            return None
        except yaml.YAMLError as e:
            print(f"✗ Error parsing YAML: {e}")
            return None
        
    # Accès aux données 
    def get(self, key_path, default=None):
        """Get a configuration value using dot notation"""
        keys = key_path.split('.')
        value = self.config

        for key in keys:
            if isinstance(value, dict) and key in value:
                value = value[key]
            else:
                return default

        return value
    # Enregistrement 
    def save_config(self, environment, config_data):
        """Save configuration to a file"""
        config_file = os.path.join(self.config_dir, f'{environment}.yaml')

        os.makedirs(self.config_dir, exist_ok=True)

        with open(config_file, 'w') as file:
            yaml.dump(config_data, file, default_flow_style=False)

        print(f"✓ Saved configuration for {environment}")


## Utilisation de la classe configManager 

In [7]:
if __name__ == '__main__':
    # Creation configMangager
    config_mgr = ConfigManager()

    # Création d'une config simple en mode développement 
    dev_config = {
        'database': {
            'host': 'localhost',
            'port': 5432,
            'name': 'dev_db'
        },
        'api': {
            'base_url': 'http://localhost:8000',
            'timeout': 30
        }
    }

    # Enregistrement 
    config_mgr.save_config('development', dev_config)

    # Chargement et utilisation 
    config_mgr.load_config('development')
    print(f"Database host: {config_mgr.get('database.host')}")
    print(f"API timeout: {config_mgr.get('api.timeout')}")


✓ Saved configuration for development
✓ Loaded configuration for development
Database host: localhost
API timeout: 30
